In [ ]:
# Import necessary libraries
from dotenv import load_dotenv
from openai import OpenAI
import os
from sentence_transformers import SentenceTransformer
from tqdm.autonotebook import tqdm


# Load environment variables from .env file
load_dotenv();

# Test Open AI embeddings

In [ ]:
from embeddings_comparison.utils.embedding_models.open_ai import OPEN_AI_EMBEDDING_MODEL, OpenAIEmbeddingModel

model = OpenAIEmbeddingModel(api_key=os.environ["OPEN_AI_KEY"], model=OPEN_AI_EMBEDDING_MODEL.TEXT_EMBEDDING_3_SMALL)
model.embed("Hello, world!")

# Test hugging face embeddings

In [ ]:
from embeddings_comparison.utils.embedding_models.hugging_face import HUGGING_FACE_EMBEDDING_MODEL, HuggingFaceEmbeddingModel

model = HuggingFaceEmbeddingModel(embedding_model_name = HUGGING_FACE_EMBEDDING_MODEL.ST_POLISH_PARAPHRASE_FROM_DISTILROBERTA)
model.embed("Hello, world!")

# Test sdadas translation transformer

In [ ]:
from embeddings_comparison.utils.translation_models.hugging_face import HUGGING_FACE_TRANSLATOR_MODEL, TranslatorEnglishToPolish

TranslatorEnglishToPolish(translation_model_name=HUGGING_FACE_TRANSLATOR_MODEL.HUGGING_FACE_TRANSLATOR).translate("Witaj świecie!")

# Test Wiki Parser

In [16]:
from embeddings_comparison.utils import wiki_parser
from pprint import pprint

pages = wiki_parser.extract_pages_from_file("data/Wikipedia-20241111162837.xml")
pprint(pages[2].sections)

[ArticleSection(title='Main', content='Wychowanie fizyczne (wf, WF) – zamierzone i świadome działanie ukierunkowane na kształtowanie postaw prosomatycznych (tj. stosunku do ciała i fizycznego funkcjonowania) oraz postaw prospołecznych (tj. stosunku do życia i innych ludzi, przekonań i wartości moralnych), a równocześnie na wspieranie rozwoju fizycznego i zdrowia oraz kształtowanie trwałego obyczaju aktywności fizycznej i troskę o wydolność fizyczną, sprawność motoryczną i prawidłową postawę ciała.'),
 ArticleSection(title='Historia wychowania fizycznego', content='Świadome wychowanie fizyczne było znane już w czasach starożytnych (spartańskie wychowanie) i traktowane na równi z wychowaniem sportowym i wojskowym; nowożytne podstawy wychowania fizycznego powstały w epoce odrodzenia i oświecenia; w XIX w. ukształtowały się niezależne systemy wychowania fizycznego: oparty na gimnastyce bez elementów sportowych oraz (głównie w USA) oparty na grach i zabawach sportowych; po II wojnie światow